In [1]:
import surrogates
import torch
import pickle

batch_size = 1000


input_data = surrogates.dir_to_dataset_list("Archive/LIGHTbeams/Trial0_Training_Data/training/240618")

file = open("Archive/240613_model_tuning/model005/starting_data/model005_rev7_cpu.pickle", "rb")
optimizer_model = pickle.load(file)
file.close()
optimizer_model.requires_grad_(False)

input_dataloader = torch.utils.data.DataLoader(input_data, batch_size=batch_size, shuffle=True)

data_iter = iter(input_dataloader)
input_batch, output_batch = next(data_iter)
input_batch.requires_grad_(True)

print(input_batch.__len__())
print(input_batch)


Not a .npy file: .DS_Store
(tensor([ 0.9055,  0.9055,  0.9055, -0.4136, -1.5403,  0.8123,  1.0000],
       requires_grad=True), tensor([-1.1203, -1.2076,  0.9738, -0.9646, -0.9335,  0.8144],
       grad_fn=<ToCopyBackward0>))
1000
tensor([[ 0.8688,  0.8688,  0.8688,  ...,  0.2199,  0.9117,  1.0000],
        [ 0.8446,  0.8446,  0.8446,  ..., -1.4352,  0.8850,  1.0000],
        [ 0.8787,  0.8787,  0.8787,  ...,  0.3570,  0.8127,  1.0000],
        ...,
        [ 0.9208,  0.9208,  0.9208,  ..., -0.6074,  0.8742,  1.0000],
        [ 0.8883,  0.8883,  0.8883,  ..., -1.3187,  0.8104,  1.0000],
        [ 0.8406,  0.8406,  0.8406,  ..., -1.4023,  0.9456,  1.0000]],
       grad_fn=<StackBackward0>)


In [64]:
print(input_data.__getitem__(0)[0])
print(input_data.out_max_list)

tensor([ 0.8974,  0.8974,  0.8974, -0.4180, -0.7581, -0.2290,  0.9200],
       requires_grad=True)
[0.07106251893096134, 0.07115837465863079, 2.6148913472005828e-08, 10369002.57741696, 10754515.254938947, 64026025.65215367, 14.999999999999964]


In [2]:
import numpy as np
#print(optimizer_model(input_data.__getitem__(700)[0]))


def prototype_solenoid_optimization(input_batch, model, strom = 0.5, l_drift = 4.9):
    
    aperture = 0.02
    input_batch.clone()
    input_tensor_0 = input_batch[:, 0:6]

    #optimizer_model.float()
    #input_tensor_0.float()

    strom_tensor = torch.ones(input_tensor_0.__len__(), dtype=torch.float32, requires_grad=True)*strom
    strom_resh = strom_tensor.view(-1, 1)
    #print(strom_tensor)

    norm_tensor = torch.tensor([0.07119202565482073, 0.07120724187081795, 2.615718890351589e-08, 10955187.108948682, 10969527.532464387, 64027733.68489398], dtype=torch.float32, requires_grad=True)

    denormalized_output = ((optimizer_model(torch.cat((input_tensor_0, strom_resh), dim=1))+1)/2)*norm_tensor
    #denormalized_output_filtered_x = denormalized_output[denormalized_output[:,0] < aperture]
    #denormalized_output_filtered_y = denormalized_output[denormalized_output[:,1] < aperture]

    #t_drift = l_drift/denormalized_output[:,5]
    t_drift = l_drift/denormalized_output[:,5]

    x_drift = t_drift*denormalized_output[:, 3]
    y_drift = t_drift*denormalized_output[:, 4]

    x_std = torch.std(x_drift)
    y_std = torch.std(y_drift)

    loss = torch.sqrt(x_std**2 + y_std**2)
    
    return loss




In [3]:
class SurrogateOptimizer(torch.nn.Module):

    def __init__(self, par, input_batch, surrogate_model):
        super().__init__()
        # register set of parameter:
        self.input_batch = input_batch
        self.surrogate_model = surrogate_model
        self.register_parameter('par', torch.nn.Parameter(par, requires_grad=True))

    def forward(self):
        # create lattice given quad strengths in k_set:
        loss = prototype_solenoid_optimization(self.input_batch, self.surrogate_model, self.par, torch.tensor(4.9, requires_grad=True))
        #self.par.register_hook(lambda grad: print(grad))
        #self.input_batch.register_hook(lambda grad_in: print(grad_in))

        return loss
    
    

In [4]:
def train_model(model, training_iter, alpha=0.1):
    history_param = [None] * training_iter  # list to save params
    history_loss = [None] * training_iter  # list to save loss



    # Use PyTorch Adam optimizer
    optimizer = torch.optim.Adam(model.parameters(recurse=False), alpha)
    
        # print the trainable parameters
    #for param in model.named_parameters():
    #    print(f'{param[0]} : {param[1]}')

    for i in range(training_iter):

        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Calc loss and backprop gradients
        # with torch.autograd.detect_anomaly():
        loss = model()  # loss is just O.F.
        loss.backward()  # gradient#
        optimizer.step()

        # print info:
        if i % 10 == 0:  # print each 100 steps
            print('Iter %d/%d - Loss: %.5f ' % (
                i + 1, training_iter, loss
            ))

        for param in model.parameters():
            history_param[i] = param.data.detach().numpy().copy()
            history_loss[i] = loss.detach().numpy().copy()

        # optimization step

    # returns params and loss for every iteration
    return np.asarray(history_param), np.asarray(history_loss)



In [5]:
strom = torch.tensor(0.8, dtype=torch.float32, requires_grad=True)

surrogate_optimizer_model = SurrogateOptimizer(strom, input_batch, optimizer_model)

params, loss = train_model(surrogate_optimizer_model, 200, .1)


Iter 1/200 - Loss: 0.11160 
Iter 11/200 - Loss: 0.11048 
Iter 21/200 - Loss: 0.10953 
Iter 31/200 - Loss: 0.10969 
Iter 41/200 - Loss: 0.10970 
Iter 51/200 - Loss: 0.10953 
Iter 61/200 - Loss: 0.10955 
Iter 71/200 - Loss: 0.10954 
Iter 81/200 - Loss: 0.10953 
Iter 91/200 - Loss: 0.10953 
Iter 101/200 - Loss: 0.10953 
Iter 111/200 - Loss: 0.10953 
Iter 121/200 - Loss: 0.10953 
Iter 131/200 - Loss: 0.10953 
Iter 141/200 - Loss: 0.10953 
Iter 151/200 - Loss: 0.10953 
Iter 161/200 - Loss: 0.10953 
Iter 171/200 - Loss: 0.10953 
Iter 181/200 - Loss: 0.10953 
Iter 191/200 - Loss: 0.10953 


In [178]:
print(strom)


tensor(0.8208, requires_grad=True)
